TODO
* Merge Tables 2,4,10 to work together
* Create loop for full season
* Add season and episode to front of data frame

Someday:
* Fix Italics:
    * Work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
    * Not work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
        
Learning: 
* Look for specific class types in BeautifulSoup. The fact that the transcript was formated as a *wikitable* made life much easier

In [1]:
# imports

#standard
import pandas as pd
#import random
from datetime import datetime
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

#Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Data Pulls
from bs4 import BeautifulSoup
import requests
import re

In [2]:
#Get Episode titles for Season 1

In [3]:
url_base = 'https://avatar.fandom.com/wiki/Avatar_Wiki:Transcripts'
response = requests.get(url_base)

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

tables = soup.find_all("table")

print(f"Found {len(tables)} tables on the page.")

Found 90 tables on the page.


In [ ]:
# Table 2: Book 1 Water
# Table 4: Book 2 Earth
# Table 10: Book 3 Fire

# if len(tables) > 0:
#     print("First table:")
#     print(tables[10].prettify())

In [5]:
book_list = [2,5,10]

In [6]:
episode_list = []

for book in book_list:
    specific_table = soup.find_all("table")[book]  # Replace 0 with the index of the table you want
    links = specific_table.find_all("a")
    for link in links:
        episode_list.append(link.get("href"))

In [7]:
episode_list_filtered = [episode for episode in episode_list if "commentary" not in episode]
episode_list_filtered

['/wiki/Transcript:The_Boy_in_the_Iceberg',
 '/wiki/Transcript:The_Avatar_Returns',
 '/wiki/Transcript:The_Southern_Air_Temple',
 '/wiki/Transcript:The_Warriors_of_Kyoshi',
 '/wiki/Transcript:The_King_of_Omashu',
 '/wiki/Transcript:Imprisoned',
 '/wiki/Transcript:Winter_Solstice,_Part_1:_The_Spirit_World',
 '/wiki/Transcript:Winter_Solstice,_Part_2:_Avatar_Roku',
 '/wiki/Transcript:The_Waterbending_Scroll',
 '/wiki/Transcript:Jet_(episode)',
 '/wiki/Transcript:The_Great_Divide',
 '/wiki/Transcript:The_Storm',
 '/wiki/Transcript:The_Blue_Spirit',
 '/wiki/Transcript:The_Fortuneteller',
 '/wiki/Transcript:Bato_of_the_Water_Tribe',
 '/wiki/Transcript:The_Deserter',
 '/wiki/Transcript:The_Northern_Air_Temple',
 '/wiki/Transcript:The_Waterbending_Master',
 '/wiki/Transcript:The_Siege_of_the_North,_Part_1',
 '/wiki/Transcript:The_Siege_of_the_North,_Part_2',
 '/wiki/Transcript:The_Avatar_State',
 '/wiki/Transcript:The_Cave_of_Two_Lovers',
 '/wiki/Transcript:Return_to_Omashu',
 '/wiki/Transcri

In [ ]:
### Get Transcript

In [8]:
#Creates Episodes URLs
base = 'https://avatar.fandom.com'
url_episode_list = []
for episode in episode_list_filtered:
    print(base + episode)
    url_episode_list.append(base + episode)

https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg
https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns
https://avatar.fandom.com/wiki/Transcript:The_Southern_Air_Temple
https://avatar.fandom.com/wiki/Transcript:The_Warriors_of_Kyoshi
https://avatar.fandom.com/wiki/Transcript:The_King_of_Omashu
https://avatar.fandom.com/wiki/Transcript:Imprisoned
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_1:_The_Spirit_World
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_2:_Avatar_Roku
https://avatar.fandom.com/wiki/Transcript:The_Waterbending_Scroll
https://avatar.fandom.com/wiki/Transcript:Jet_(episode)
https://avatar.fandom.com/wiki/Transcript:The_Great_Divide
https://avatar.fandom.com/wiki/Transcript:The_Storm
https://avatar.fandom.com/wiki/Transcript:The_Blue_Spirit
https://avatar.fandom.com/wiki/Transcript:The_Fortuneteller
https://avatar.fandom.com/wiki/Transcript:Bato_of_the_Water_Tribe
https://avatar.fandom.com/wiki/Transcript:The_Des

In [9]:
for episode_number, url in enumerate(url_episode_list):
    
    split_text = url.split(':', 2)
    episode_title = split_text[2]

    episode_number += 1
    print(episode_number)
    print(episode_title)
    print(url)
    
    response = requests.get(url)
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the tables with the class "wikitable"
    wikitable_tables = soup.find_all("table", class_="wikitable")

# Print the count of wikitables
    print(f"Number of wikitables: {len(wikitable_tables)}")
    print()

    
# Edge Cases
1
2
15 Deleted Scence
35 Tales from Ba Sing Se 

1
The_Boy_in_the_Iceberg
https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg
Number of wikitables: 2

2
The_Avatar_Returns
https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns
Number of wikitables: 2

3
The_Southern_Air_Temple
https://avatar.fandom.com/wiki/Transcript:The_Southern_Air_Temple
Number of wikitables: 1

4
The_Warriors_of_Kyoshi
https://avatar.fandom.com/wiki/Transcript:The_Warriors_of_Kyoshi
Number of wikitables: 1

5
The_King_of_Omashu
https://avatar.fandom.com/wiki/Transcript:The_King_of_Omashu
Number of wikitables: 1

6
Imprisoned
https://avatar.fandom.com/wiki/Transcript:Imprisoned
Number of wikitables: 1

7
Winter_Solstice,_Part_1:_The_Spirit_World
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_1:_The_Spirit_World
Number of wikitables: 1

8
Winter_Solstice,_Part_2:_Avatar_Roku
https://avatar.fandom.com/wiki/Transcript:Winter_Solstice,_Part_2:_Avatar_Roku
Number of wikitables: 1

9
The_Waterbending_Scroll
https://avatar.fandom.com/

In [ ]:
Test Case

In [ ]:
#IDentify problem of if intro is said

In [ ]:
# Full Season Loop

url = 'https://avatar.fandom.com/wiki/Transcript:The_Siege_of_the_North,_Part_2'
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

print(tables)

# data = []

# if len(tables) > 1:
#     second_table = tables[1]  # Index 1 corresponds to 2nd Table which holds the transcript
# # Find all rows within the 2nd Table that holds the transcript
#     rows = second_table.select("tbody tr")

# for row in rows:
#     character_cell = row.find('th')  # Find the <th> for character names
#     dialogue_cell = row.find('td')    # Find the <td> for character's dialogue

#     if character_cell and dialogue_cell:  # Ensure both character & dialogue are present (thus skipping pure screen directions)
#         character = character_cell.get_text(strip=True)

#         # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
#         for italics in dialogue_cell.find_all('i'):
#             # Get the text inside <i>, append '123' and replace the <i> with this text
#             italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
#             italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

#         dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

#         data.append({'character': character, 'dialogue': dialogue})

#         df = pd.DataFrame(data)

# #Cleaning
# df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
# df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
# df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

# #Word Count
# df['word_count'] = df['dialogue'].str.split().apply(len)

# #Episode 
# df['episode'] = episode_number
# df['espisode_title'] = episode_title 
            
# #     print(episode_title)
    
# #     transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
df

In [ ]:
# Full Season Loop

transcript_df = pd.DataFrame()

for episode_number, url in enumerate(url_episode_list):
    
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1
    print(episode_number)
    print(episode_title)
    print(url)
    print()
    
    response = requests.get(url)
    ep_soup = BeautifulSoup(response.content, "html.parser")
    tables = ep_soup.select("table.wikitable")

    data = []
    
    if len(tables) > 1:
        second_table = tables[1]  # Index 1 corresponds to 2nd Table which holds the transcript
    # Find all rows within the 2nd Table that holds the transcript
        rows = second_table.select("tbody tr")
    
    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')    # Find the <td> for character's dialogue
    
        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present (thus skipping pure screen directions)
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
            for italics in dialogue_cell.find_all('i'):
                # Get the text inside <i>, append '123' and replace the <i> with this text
                italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text
            
            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics
            
            data.append({'character': character, 'dialogue': dialogue})

            df = pd.DataFrame(data)
            
    #Cleaning
    df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
    df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
    df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

    #Word Count
    df['word_count'] = df['dialogue'].str.split().apply(len)

    #Episode 
    df['episode'] = episode_number
    df['espisode_title'] = episode_title 
            
#     print(episode_title)
    
#     transcript_df = pd.concat([transcript_df, df], ignore_index=True)

In [ ]:
df.head()

In [ ]:
test = transcript_df[transcript_df['episode'] == 7]
test

In [ ]:
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None) 

from tabulate import tabulate

for episode in range(1,21):
    test = transcript_df[transcript_df['episode'] == episode]
    print(test)
    #print(tabulate(test, headers='keys', tablefmt='pretty'))
    print()


In [ ]:
transcript_df.head(20)

In [ ]:
transcript_df.tail(20)

In [ ]:
#First Episode
episode_base = url_episode_list[0]
response = requests.get(episode_base)
ep_soup = BeautifulSoup(response.content, "html.parser")
#tables = ep_soup.find_all("table")

In [ ]:
tables = ep_soup.select("table.wikitable")
data = []

# Step 4: Check if there are at least 2 tables and select the second one
if len(tables) > 1:
    second_table = tables[1]  # Index 1 corresponds to the second table
    # Now, find all rows within this second table's tbody
    rows = second_table.select("tbody tr")
    
    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')    # Find the <td> for dialogue
    
        if character_cell and dialogue_cell:  # Ensure both are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
            for italics in dialogue_cell.find_all('i'):
                # Get the text inside <i>, append '123' and replace the <i> with this text
                italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text
            
            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics
            
            data.append({'character': character, 'dialogue': dialogue})

            df = pd.DataFrame(data)
            
            #Cleaning
            df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
            df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
            df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()
            
            #Word Count
            df['word_count'] = df['dialogue'].str.split().apply(len)

In [ ]:
pd.set_option('display.max_colwidth', None) 
df.head(20)

In [ ]:
pd.set_option('display.max_colwidth', None) 
df.tail(20)

In [ ]:
# Table 1 - Intro
if len(tables) > 0:
    print("Second table:")
    print()
    print(tables[1].prettify())